# Import libraries
These libraries will be used for our URL_classification project.

In [1]:
import datetime
import csv
import nltk
import numpy as np
import pandas as pd
import ast
from urllib.request import urlopen
from bs4 import BeautifulSoup
import os.path

print(datetime.datetime.now().time())

14:23:30.843081


Use this command if you have any errors on importing nltk library. It will open a nltk meniu with download and update options. If it's still missing some libraries, it needs to install manually by writing nltk.download('library name') where library name is missing library name which asserts error message.

In [2]:
nltk.download('stopwords')
nltk.download('words')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/domantas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /home/domantas/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package punkt to /home/domantas/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def conctruct_dataset():
    file = 'URL-categorization-DFE.csv'
    df = pd.read_csv(file)[['main_category', 'main_category:confidence', 'url']]
    df = df[(df['main_category'] != 'Not_working') & (df['main_category:confidence'] > 0.5)]

    char_blacklist = list(chr(i) for i in range(32, 127) if i <= 64 or i >= 91 and i <= 96 or i >= 123)
    stopwords = nltk.corpus.stopwords.words('english')
    stopwords.extend(char_blacklist)
    language_whitelist = ['en']
    english_vocab = set(w.lower() for w in nltk.corpus.words.words())
    blacklist_domain = ['.it', '.ru', '.cn', '.jp', '.tw', '.de', '.pl', '.fr', '.hu', '.bg', '.nl']

    df = df[~df['url'].str.endswith(tuple(blacklist_domain))]
    df['tokenized_words'] = ''
    
    counter = 0
    for i, row in df.iterrows():
        counter += 1
        if counter >= 50:
            break
        print("{}, {}/{}".format(row['url'], counter, len(df)))

        try:
            html = urlopen('http://' + row['url'], timeout=1).read()
        except:
            continue

        soup = BeautifulSoup(html, "html.parser")
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
        filter_text = " ".join(w for w in nltk.word_tokenize(text) \
             if w.lower() in english_vocab)

        tokens = nltk.word_tokenize(filter_text)

        allWordExceptStopDist = nltk.FreqDist(
            w.lower() for w in tokens if w not in stopwords and len(w) >= 3 and w[0] not in char_blacklist)

        all_words = [i for i in allWordExceptStopDist]

        if len(all_words) > 0:
            continue

        df.at[i, 'tokenized_words'] = all_words
        
    df = df[df['tokenized_words'] != '']

In [4]:
def train_machine():
    char_blacklist = list(chr(i) for i in range(32, 127) if i <= 64 or i >= 91 and i <= 96 or i >= 123)
    stopwords = nltk.corpus.stopwords.words('english')
    stopwords.extend(char_blacklist)
    df = pd.read_csv('cleaned_data.csv')
    
    top = 50
    words_frequency = {}
    for category in set(df['main_category'].values):
        all_words = []
        for row in df[df['main_category'] == category]['tokenized_words'].tolist():
            for word in ast.literal_eval(row):
                all_words.append(word)
        allWordExceptStopDist = nltk.FreqDist(
            w.lower() for w in all_words if w not in stopwords and len(w) >= 3 and w[0] not in char_blacklist)

        most_common = allWordExceptStopDist.most_common(top)
        words_frequency[category] = most_common
    
    for category in set(df['main_category'].values):
        words_frequency[category] = [word for word, number in words_frequency[category]]
        
    from collections import Counter
    
    features = np.zeros(df.shape[0] * top).reshape(df.shape[0], top)
    labels = np.zeros(df.shape[0])
    counter = 0
    for i, row in df.iterrows():
        c = [word for word, word_count in Counter(ast.literal_eval(row['tokenized_words'])).most_common(top)]
        labels[counter] = list(set(df['main_category'].values)).index(row['main_category'])
        for word in c:
            if word in words_frequency[row['main_category']]:
                features[counter][words_frequency[row['main_category']].index(word)] = 1
        counter += 1
        
    return labels, features

In [8]:
def no_filter_data():
    file = 'URL-categorization-DFE.csv'
    df = pd.read_csv(file)[['main_category', 'main_category:confidence', 'url']]
    df = df[(df['main_category'] != 'Not_working') & (df['main_category:confidence'] > 0.5)]
    df['tokenized_words'] = ''
    
    counter = 0
    for i, row in df.iterrows():
        counter += 1
        print("{}, {}/{}".format(row['url'], counter, len(df)))

        try:
            html = urlopen('http://' + row['url'], timeout=1).read()
        except:
            continue

        soup = BeautifulSoup(html, "html.parser")
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk.lower() for chunk in chunks if chunk)
        tokens = nltk.word_tokenize(text)
        
        df.at[i, 'tokenized_words'] = tokens if len(tokens) > 0 else ''
        
    df = df[df['tokenized_words'] != '']
    return df

In [10]:
# if os.path.isfile("cleaned_data.csv"):
#     labels, features = train_machine()
# else:
#     conctruct_dataset()
#     labels, features = train_machine()
if not os.path.isfile("Datasets/full_data.csv"):
    df = no_filter_data()

In [11]:
df.shape

NameError: name 'df' is not defined

In [14]:
from sklearn.metrics import accuracy_score
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(features)

from sklearn.utils import shuffle
X, X_sparse, y = shuffle(features, X_sparse, labels, random_state=0)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [18]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)
score = lr.score(X_test, y_test)
print(predictions)
print(score)

[12. 12. 15. ...  1.  1. 18.]
0.42168674698795183


In [16]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X_train, y_train)
predictions = dtc.predict(X_test)
score = dtc.score(X_test, y_test)
print(predictions)
print(score)

[ 9. 24.  4. ...  1. 19. 18.]
0.2754350736278447


In [17]:
from sklearn import svm
svm = svm.SVC()
svm.fit(X_train, y_train)
predictions = svm.predict(X_test)
score = svm.score(X_test, y_test)
print(predictions)
print(score)

[12. 12. 15. ...  1.  1. 18.]
0.3989290495314592
